In [79]:
from osgeo import gdal
import re
from typing import Union
import h5py
import numpy as np
import os


wavelengths = [
    444,
    475,
    531,
    560,
    650,
    668,
    705,
    717,
    740,
    862
]  # In nm, each index corresponds to each band


fwhm = [
    28,
    32,
    14,
    27,
    16,
    14,
    10,
    12,
    18,
    57
]


def create_h5_file_from_dict(data, h5_file, group_name="/"):
    """
    Recursively create groups and datasets in an HDF5 file from a dictionary.

    Parameters:
        data (dict): Input dictionary.
        h5_file (h5py.File): HDF5 file object.
        group_name (str): Name of the current group (default: "/").
    """
    for key, value in data.items():
        # If the value is a dictionary, create a group
        if isinstance(value, dict):
            subgroup_name = f"{group_name}/{key}"
            _ = h5_file.create_group(subgroup_name)
            create_h5_file_from_dict(value, h5_file, subgroup_name)
        # If the value is not a dictionary, create a dataset
        else:
            dataset_name = f"{group_name}/{key}"
            if os.path.basename(group_name) == 'Coordinate_System':
                # Convert value to string before saving
                dataset = h5_file.create_dataset(dataset_name, data=str(value), dtype=h5py.string_dtype())
            else:
                dataset = h5_file.create_dataset(dataset_name, data=value)

            if key in ['Wavelength', 'FWHM']:
                dataset.attrs['Units'] = 'nanometers'


def remove_non_numeric(input_string):
    # Use regex to replace non-numeric characters with an empty string
    return re.sub(r'\D', '', input_string)


def tiff_to_h5(reflectance_tiff_path: str, slope_data: Union[str, np.array],
               aspect_data: Union[str, np.array], sensor_zenith_data: Union[str, np.array],
               sensor_azimuth_data: Union[str, np.array], solar_zenith: float, solar_azimuth: float):

    reflectance = gdal.Open(reflectance_tiff_path)

    if isinstance(slope_data, str):
        slope_data = gdal.Open(slope_data).ReadAsArray()
    if isinstance(aspect_data, str):
        aspect_data = gdal.Open(aspect_data).ReadAsArray()
    if isinstance(sensor_azimuth_data, str):
        sensor_zenith_data = gdal.Open(sensor_zenith_data).ReadAsArray()
    if isinstance(sensor_azimuth_data, str):
        sensor_azimuth_data = gdal.Open(sensor_azimuth_data)

    proj = reflectance.GetProjection()
    geo_transform = reflectance.GetGeoTransform()

    spatial_ref = gdal.osr.SpatialReference()
    spatial_ref.ImportFromWkt(proj)
    epsg_code = spatial_ref.GetAuthorityCode(None)
    utm_zone = int(remove_non_numeric(proj.split("UTM zone ")[1].split(",")[0]))
    map_info_string = f"UTM, 1.000, 1.000, {geo_transform[0]:.3f}, {geo_transform[3]:.3f}, {geo_transform[1]:.3f}, {geo_transform[5]:.3f}, {utm_zone}, North, WGS-84, units=Meters, 0",

    h5_data = {
        'NIWO': {
            'Reflectance': {
                'Metadata': {
                    'Coordinate_System': {
                        'Coordinate_System_String': proj,
                        'EPSG Code': epsg_code,
                        'Map_Info': map_info_string,
                        'Proj4': spatial_ref.ExportToProj4()
                    },
                    # below I chnaged path length from np.zeros to np.ones()
                    "Ancillary_Imagery/Path_Length": np.ones(reflectance.ReadAsArray().shape[1:]),
                    "to-sensor_Azimuth_Angle": sensor_azimuth_data,
                    "to-sensor_Zenith_Angle": sensor_zenith_data,
                    "Logs/Solar_Azimuth_Angle": solar_azimuth,
                    "Logs/Solar_Zenith_Angle": solar_zenith,
                    "Ancillary_Imagery/Slope": slope_data,
                    "Ancillary_Imagery/Aspect": aspect_data,
                    "Spectral_Data": {
                        'FWHM': fwhm,
                        'Wavelength': wavelengths
                    }
                },
                'Reflectance_Data': np.transpose(reflectance.ReadAsArray()[:10, :, :], axes=(1, 2, 0))
            }
        }
    }

    with h5py.File('NEON_D13_NIWO_test' + os.path.basename(reflectance_tiff_path).replace('.tif', '.h5'), "w") as hdf_file:
        create_h5_file_from_dict(h5_data, hdf_file)

In [80]:
import ephem
import rasterio
import numpy as np
from rasterio import Affine as A
from rasterio.warp import reproject, Resampling
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from osgeo import gdal
import seaborn as sns
import rasterio
from pyproj import Proj, transform
from rasterio.transform import from_origin
import ephem
import datetime
import math
import warnings

def pixel_to_coord(file_path):
     with rasterio.open(file_path) as src:
            band1 = src.read(1)
            print('Band1 has shape', band1.shape)
            height = band1.shape[0]
            width = band1.shape[1]
            cols, rows = np.meshgrid(np.arange(width), np.arange(height))
            xs, ys = rasterio.transform.xy(src.transform, rows, cols)
            eastings= np.array(xs)
            northings = np.array(ys)
            print('eastings shape', eastings.shape)
            p1 = Proj(src.crs)
            p2 = Proj(proj='latlong', datum='WGS84')
            lons, lats = transform(p1, p2, eastings, northings)
     return lons, lats, cols, rows,


file_path = r'/home/jovyan/data-store/cross-sensor-cal/data_set2/aligned_orthomosaic.tif'
longitudes, latitudes, cols, rows = pixel_to_coord(file_path)

# # # Function to convert lat/lon to row/col
# # def latlon_to_rowcol(transform, lat, lon):
# #     col, row = ~transform * (lon, lat)
# #     return int(row), int(col)

# Convert the date and time to UTC. The time given is 2:34 PM, which is 14:34 in 24-hour format
date_time_str = '2023-08-01 14:34:00'


with rasterio.open(file_path) as src:
    # Get the affine transform for the raster
    transform = src.transform
    
    # Create arrays to hold the azimuth and zenith values
    azimuth = np.zeros((src.height, src.width), dtype=np.float32)
    zenith = np.zeros((src.height, src.width), dtype=np.float32)
    
    # Assume a date and time for the Sun position calculation
    observer = ephem.Observer()
    observer.date = ephem.date(date_time_str)
    
    # Iterate over each pixel in the raster
    for row in range(latitudes.shape[0]):
        for col in range(latitudes.shape[1]):
            #lon, lat = pixel_to_coord(transform, row, col)
            observer.lat, observer.lon = latitudes[row,col], longitudes[row,col]
            
            sun = ephem.Sun(observer)
            
            # Convert azimuth and altitude (zenith angle is 90 - altitude) to degrees
            #az = np.degrees(sun.az)
            az = sun.az
            alt = np.degrees(sun.alt)
            zen = 90 - sun.alt
            
            azimuth[row, col] = az
            zenith[row, col] = zen



# with rasterio.open(file_path) as src:
#     # Get the affine transform for the raster
#     transform = src.transform
    
#     # Create arrays to hold the azimuth and zenith values
#     azimuth = np.zeros((src.height, src.width), dtype=np.float32)
#     zenith = np.zeros((src.height, src.width), dtype=np.float32)
    
#     # Assume a date and time for the Sun position calculation
#     observer = ephem.Observer()
#     observer.date = ephem.date(date_time_str)
    
#     # Iterate over each pixel in the raster
#     for row in range(src.height):
#         for col in range(src.width):
#             #lon, lat = pixel_to_coord(transform, row, col)
#             observer.lat, observer.lon = str(lat), str(lon)
            
#             sun = ephem.Sun(observer)
            
#             # Convert azimuth and altitude (zenith angle is 90 - altitude) to degrees
#             az = np.degrees(sun.az)
#             alt = np.degrees(sun.alt)
#             zen = 90 - alt
            
#             azimuth[row, col] = az
#             zenith[row, col] = zen

Band1 has shape (1018, 672)
eastings shape (1018, 672)


/tmp/ipykernel_536/3404857961.py:33: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lons, lats = transform(p1, p2, eastings, northings)


In [83]:
longitudes

array([[-105.54106897, -105.54106604, -105.5410631 , ..., -105.53910866,
        -105.53910573, -105.5391028 ],
       [-105.54106895, -105.54106602, -105.54106309, ..., -105.53910864,
        -105.53910571, -105.53910278],
       [-105.54106893, -105.541066  , -105.54106307, ..., -105.53910862,
        -105.53910569, -105.53910276],
       ...,
       [-105.5410509 , -105.54104797, -105.54104504, ..., -105.53909066,
        -105.53908773, -105.5390848 ],
       [-105.54105088, -105.54104795, -105.54104502, ..., -105.53909064,
        -105.53908771, -105.53908478],
       [-105.54105086, -105.54104793, -105.541045  , ..., -105.53909062,
        -105.53908769, -105.53908476]])

In [84]:
latitudes

array([[40.03632017, 40.03632018, 40.0363202 , ..., 40.03632931,
        40.03632932, 40.03632934],
       [40.03631792, 40.03631793, 40.03631795, ..., 40.03632706,
        40.03632707, 40.03632708],
       [40.03631567, 40.03631568, 40.03631569, ..., 40.0363248 ,
        40.03632482, 40.03632483],
       ...,
       [40.03403405, 40.03403406, 40.03403408, ..., 40.03404319,
        40.0340432 , 40.03404321],
       [40.0340318 , 40.03403181, 40.03403182, ..., 40.03404093,
        40.03404095, 40.03404096],
       [40.03402955, 40.03402956, 40.03402957, ..., 40.03403868,
        40.0340387 , 40.03403871]])

In [85]:
azimuth

array([[5.0587783, 5.058781 , 5.0587845, ..., 5.0607386, 5.060742 ,
        5.060745 ],
       [5.0587783, 5.0587816, 5.0587845, ..., 5.0607386, 5.060742 ,
        5.060745 ],
       [5.0587783, 5.0587816, 5.0587845, ..., 5.060739 , 5.060742 ,
        5.060745 ],
       ...,
       [5.0587964, 5.0587997, 5.0588026, ..., 5.0607567, 5.0607595,
        5.060763 ],
       [5.0587964, 5.0587997, 5.0588026, ..., 5.0607567, 5.06076  ,
        5.060763 ],
       [5.0587964, 5.0587997, 5.0588026, ..., 5.0607567, 5.06076  ,
        5.060763 ]], dtype=float32)

In [86]:
zenith

array([[89.53681 , 89.536804, 89.536804, ..., 89.53545 , 89.535446,
        89.535446],
       [89.53681 , 89.536804, 89.536804, ..., 89.53545 , 89.535446,
        89.535446],
       [89.53681 , 89.536804, 89.536804, ..., 89.53545 , 89.535446,
        89.535446],
       ...,
       [89.53684 , 89.53684 , 89.53684 , ..., 89.53549 , 89.53549 ,
        89.535484],
       [89.53684 , 89.53684 , 89.53684 , ..., 89.53549 , 89.53549 ,
        89.535484],
       [89.53684 , 89.53684 , 89.53684 , ..., 89.53549 , 89.53549 ,
        89.535484]], dtype=float32)

In [71]:
np.mean(latitudes)

40.03517944314455

In [72]:
np.mean(longitudes)

-105.54007684654168

In [89]:
tiff_to_h5('/home/jovyan/data-store/cross-sensor-cal/data_set2/aligned_orthomosaic.tif', '/home/jovyan/data-store/cross-sensor-cal/data_set2/slope_smooth_slope.tif', '/home/jovyan/data-store/cross-sensor-cal/data_set2/aspect_smooth_aspect.tif', zenith, azimuth, solar_zenith=zenith, solar_azimuth=azimuth)
print("6")

6


In [88]:
azimuth

array([[5.0587783, 5.058781 , 5.0587845, ..., 5.0607386, 5.060742 ,
        5.060745 ],
       [5.0587783, 5.0587816, 5.0587845, ..., 5.0607386, 5.060742 ,
        5.060745 ],
       [5.0587783, 5.0587816, 5.0587845, ..., 5.060739 , 5.060742 ,
        5.060745 ],
       ...,
       [5.0587964, 5.0587997, 5.0588026, ..., 5.0607567, 5.0607595,
        5.060763 ],
       [5.0587964, 5.0587997, 5.0588026, ..., 5.0607567, 5.06076  ,
        5.060763 ],
       [5.0587964, 5.0587997, 5.0588026, ..., 5.0607567, 5.06076  ,
        5.060763 ]], dtype=float32)

In [77]:
from math import sin, cos, radians, degrees, acos, asin

# Input parameters
latitude = 40.03505796360895  # degrees
longitude = -105.53999928775333  # degrees
day_of_year = 226  # example: June 21
local_time = 14.24  # noon in hours
timezone_offset = -7  # UTC offset for PDT

# Solar declination
declination = -23.44 * cos(radians(360 * (day_of_year + 10) / 365))

# Equation of Time (EoT)
eot = 7.5 * sin(radians(360 * (day_of_year - 81) / 365))

# Local Solar Time (LST)
lst = local_time + (longitude / 15) + eot - timezone_offset

# Hour Angle (H)
hour_angle = 15 * (lst - 12)

# Solar Zenith Angle
cos_zenith = (sin(radians(latitude)) * sin(radians(declination)) +
              cos(radians(latitude)) * cos(radians(declination)) * cos(radians(hour_angle)))
zenith = degrees(acos(cos_zenith))

# Solar Azimuth Angle
cos_azimuth = (sin(radians(declination)) - sin(radians(latitude)) * cos(radians(zenith))) / \
              (cos(radians(latitude)) * sin(radians(zenith)))
azimuth = degrees(acos(cos_azimuth))

# Adjust azimuth depending on hour angle
if hour_angle > 0:
    azimuth = 360 - azimuth

print(f"Solar Zenith Angle: {zenith:.2f}°")
print(f"Solar Azimuth Angle: {azimuth:.2f}°")


Solar Zenith Angle: 88.90°
Solar Azimuth Angle: 287.69°


In [76]:
zenith


88.89764248672688

In [ ]:
NEON_D13_NIWO_ortho_mrs2.h5